In [1]:
import os

os.chdir("/work")

# %% 準備
import pandas as pd
from utils.logger import app_logger
from utils.config_loader import get_template_config
from logic.manage.utils.csv_loader import load_all_filtered_dataframes
from logic.manage.utils.load_template import load_master_and_template
from IPython.display import display
import re
from logic.manage.factory_report import process
from logic.manage.utils.excel_tools import create_label_rows_generic, sort_by_cell_row

2025-04-25 10:33:53,572 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ factory_report.py の process 関数を登録しました
2025-04-25 10:33:53,579 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ balance_sheet.py の process 関数を登録しました
2025-04-25 10:33:53,595 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ average_sheet.py の process 関数を登録しました
2025-04-25 10:33:53,602 [INFO] (__init__.py:24) [fb6959af9847/root] ✅ management_sheet.py の process 関数を登録しました


In [2]:
from logic.manage.processors.factory_report_yuuka import process_yuuka
from logic.manage.processors.factory_report_shobun import process_shobun

In [3]:
# 表示ラベルマップ（処理対象名として使う）
csv_label_map = {"yard": "ヤード一覧", "shipping": "出荷一覧", "receive": "受入一覧"}

debug_shipping = "/work/data/output/debug_shipping.parquet"
debug_yard = "/work/data/output/debug_yard.parquet"

dfs = {
    "shipping": pd.read_parquet(debug_shipping),
    "yard": pd.read_parquet(debug_yard),
}  # テスト用CSV
# dfs
df_shipping = dfs["shipping"]
df_yard = dfs["yard"]

In [4]:
df_shipping

,伝票日付,出荷番号,取引先名,業者CD,業者名,現場CD,現場名,品名,正味重量,数量,単位名,単価,金額,運搬業者名,伝票区分名
0,2025-03-31,10499,■I ･ S ｱｲ･ｴｽ,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),13870,"13,870",kg,22,"305,140",ｵﾈｽﾄ,支払
1,2025-03-31,10554,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),1990,"1,990",kg,31.5,"62,685",東京産資源,売上
2,2025-03-31,10553,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3390,"3,390",kg,31.5,"106,785",東京産資源,売上
3,2025-03-31,10552,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3000,"3,000",kg,31.5,"94,500",東京産資源,売上
4,2025-03-31,10551,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3810,"3,810",kg,31.5,"120,015",東京産資源,売上
5,2025-03-31,10550,東京産資源,8425,東京産資源,1,【ﾔｰﾄﾞ】,GB(ｷﾞﾛA),3310,"3,310",kg,31.5,"104,265",東京産資源,売上
6,2025-03-31,10549,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(B),0,"10,590",kg,1,"10,590",ｵﾈｽﾄ,支払
7,2025-03-31,10548,ADVANCE,8327,丸源起業,0,丸源起業ひかりｴｺｽﾃｰｼｮﾝ,混合廃棄物(A),0,"13,850",kg,1,"13,850",ｵﾈｽﾄ,支払
8,2025-03-31,10547,東京ﾎﾞｰﾄﾞ工業,5238,東京ﾎﾞｰﾄﾞ工業,0,東京ﾎﾞｰﾄﾞ工場新木場ﾘｻｲｸﾘﾝｸﾞ工場,木くず,2050,"2,050",kg,7,"14,350",■ｴｺﾗｲﾝ,支払
9,2025-03-31,10546,宮崎,747,宮崎,1,宮崎新木場リサイクルセンター,ﾀﾞﾝﾎﾞｰﾙ,250,250,kg,14,"3,500",■ｴｺﾗｲﾝ,売上


In [5]:
# --- ① マスターCSVの読み込み ---
config = get_template_config()["factory_report"]
master_path = config["master_csv_path"]["shobun"]
master_csv = load_master_and_template(master_path)

In [6]:

# --- 初期処理 ---
df_shipping = df_shipping.copy()
df_shipping["業者CD"] = df_shipping["業者CD"].astype(str)
marugen_num = "8327"

# --- 丸源処理 ---
df_marugen = df_shipping[df_shipping["業者CD"] == marugen_num].copy()
filtered_marugen = df_marugen[df_marugen["品名"].isin(master_csv["品名"])]
agg_marugen = filtered_marugen.groupby(["業者CD", "品名"], as_index=False)[
    "正味重量"
].sum()

agg_marugen


,業者CD,品名,正味重量
0,8327,混合廃棄物(A),13870
1,8327,混合廃棄物(B),10600


In [7]:

# --- その他業者処理 ---
df_others = df_shipping[df_shipping["業者CD"] != marugen_num]
agg_others = df_others.groupby("業者CD", as_index=False)["正味重量"].sum()
agg_others


,業者CD,正味重量
0,1,3750
1,5238,10420
2,747,440
3,8215,13280
4,8334,12710
5,8339,2780
6,8340,2680
7,8425,16000
8,8463,6750


In [8]:
# --- 統合・整形 ---
aggregated = pd.concat([agg_others, agg_marugen], ignore_index=True)
aggregated.rename(columns={"業者CD": "業者CD", "品名": "品名"}, inplace=True)
aggregated



,業者CD,正味重量,品名
0,1,3750,NaN
1,5238,10420,NaN
2,747,440,NaN
3,8215,13280,NaN
4,8334,12710,NaN
5,8339,2780,NaN
6,8340,2680,NaN
7,8425,16000,NaN
8,8463,6750,NaN
9,8327,13870,混合廃棄物(A)


In [9]:

master_csv["値"] = pd.to_numeric(master_csv["値"], errors="coerce").fillna(0)

master_csv

,業者CD,業者名,品名,セル,値
0,8339.0,共同土木,NaN,C30,0.0
1,5238.0,東京ボード,NaN,F30,0.0
2,8340.0,木材開発,NaN,I30,0.0
3,8085.0,松岡環境開発,NaN,L30,0.0
4,6421.0,アクトリー,NaN,C32,0.0
5,8337.0,丸幸,NaN,F32,0.0
6,8215.0,杉田建材,NaN,I32,0.0
7,8336.0,新潟環境,NaN,L32,0.0
8,8327.0,丸源企業（ S ）,混合廃棄物(S),C34,0.0
9,8327.0,丸源企業（ A ）,混合廃棄物(A),F34,0.0


/work/utils/data_cleaning.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['8339' '5238' '8340' '8085' '6421' '8337' '8215' '8336' '8327' '8327'
 '8327' '8327' '705' '1185' '8076' '8077' '5199' '8334' '6797' '8338'
 '8341' '8080' '349' '6786' '5227' '8329' '8360' '8363' '8398' '8679'
 '339' '8082']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[valid, col] = df.loc[valid, col].apply(lambda x: str(int(float(x))))


In [11]:
aggregated

,業者CD,正味重量,品名
0,1,3750,NaN
1,5238,10420,NaN
2,747,440,NaN
3,8215,13280,NaN
4,8334,12710,NaN
5,8339,2780,NaN
6,8340,2680,NaN
7,8425,16000,NaN
8,8463,6750,NaN
9,8327,13870,混合廃棄物(A)


In [12]:

updated_master = master_csv.merge(aggregated, on=["業者CD", "品名"], how="left")
updated_master

,業者CD,業者名,品名,セル,値,正味重量
0,8339,共同土木,NaN,C30,0.0,2780.0
1,5238,東京ボード,NaN,F30,0.0,10420.0
2,8340,木材開発,NaN,I30,0.0,2680.0
3,8085,松岡環境開発,NaN,L30,0.0,NaN
4,6421,アクトリー,NaN,C32,0.0,NaN
5,8337,丸幸,NaN,F32,0.0,NaN
6,8215,杉田建材,NaN,I32,0.0,13280.0
7,8336,新潟環境,NaN,L32,0.0,NaN
8,8327,丸源企業（ S ）,混合廃棄物(S),C34,0.0,NaN
9,8327,丸源企業（ A ）,混合廃棄物(A),F34,0.0,13870.0


In [ ]:

updated_master["正味重量"] = updated_master["正味重量"].fillna(0)
updated_master["値"] += updated_master["正味重量"]
updated_master.drop(columns=["正味重量"], inplace=True)
